In [ ]:
#####  Routines for Pre-processing (Data Wrangling/Data Cleaning) any dataset for further Data Analysis

##### IMPORTANT !!! --> Run each block below indivdually, so you can make decisions according to the results 
#                       to decide to continue with the next block.  


####################################  Run this block to load dataset and show data (feature) types
file_name = 'name of file'  #<--------inform file here  
path = 'file path'          #<--------inform path here 

import pandas as pd
import numpy as np
pd.set_option('display.max_columns',None)     # Optional 
pd.set_option('display.max_rows',None)      # Optional
df =pd.read_csv(path+file_name)
df.dtypes

####################################    Run this block to show first 10 rows of dataset
df.head(10)

####################################   Run this block to show a summary (distribution) of data (numerical/categorical features)
df.describe(include='all')


###################################   Run this block to show all features containing missing values, and their percentage proportion
allFeatures= []
for feature in list(df.columns): 
    if  len(df[feature].isnull().value_counts()) == 1: # no missing values for this feature
        allFeatures.append([feature,0,100]) 
    else:   
        allFeatures.append([feature, df[feature].isnull().value_counts()[True]*100/len(df),df[feature].isnull().value_counts()[False]*100/len(df)])

dfShowMissingValues = pd.DataFrame(allFeatures,columns=['features','missing values %','identified values%']).sort_values('missing values %')
dfShowMissingValues.reset_index()

###################################   Run this block to print all the features to copy-paste below only the ones to be kept
print('Get from the list below only the features to be kept:')
list(dfShowMissingValues.features)


###################################   Run this block to define here the columns to keep 
columnsToKeep = [] # <----Copy inside this brackets (from what was printed above), only the features to be kept
print('Features that will be kept:')
columnsToKeep

##################################   Run this block to show columns that were kept, but still have missing values (grouped by data type)
dfcolumnsWithMissingValuesToKeep = pd.Series(df[columnsToKeep].isnull().any().index)[list(df[columnsToKeep].isnull().any())]

dfMissing = df[dfcolumnsWithMissingValuesToKeep]

print('These are the features kept that are still with missing values:')

for featureType in dfMissing.dtypes.unique():
    print('\n','-'*40,'\n********',featureType,'type Features with missing values'.upper(),'\n','-'*40) 
    print(dfMissing.dtypes[dfMissing.dtypes == featureType],'\n','-'*40) 
    
    for feature in dfMissing.dtypes.index[dfMissing.dtypes == featureType]:
        print(dfMissing[feature].isnull().value_counts(),'\n','-'*40) 

dfFinal = df.copy()[columnsToKeep]


#////////////////////// Run this block to replace missing values of ALL numerical features of type (--specify type--) by the average of their columns
typeOfNumerical = 'new type' #<------specify type the new type-- Ex: float64

print('\n********',typeOfNumerical,'type Features containing missing values before - values NOW','\n','-'*40) 
for feature in list(dfFinal.dtypes[dfFinal.dtypes == typeOfNumerical].index):
    dfFinal[feature].replace(np.nan,dfFinal[feature].mean(),inplace=True)
    print(dfFinal[feature].isnull().value_counts(),'\n','-'*40) 


# ////////////////////// Run this block to replace missing values of a categorical feature (--specify column--) by the most frequent value in the column
categoricalColumn = 'name' #<------specify name of categorical column here----

dfFinal[categoricalColumn].replace(np.nan,df.describe(include='all')[categoricalColumn].top,inplace=True)
print('\n******** MISSING VALUES OF CATEGORICAL COLUMN',categoricalColumn,'REPLACED WITH THE MOST FREQUENT FOUND ON COLUMN:',df.describe(include='all')[categoricalColumn].top,'\n','-'*40) 
print('\n********',categoricalColumn,'column containing missing values before - values NOW','\n','-'*40) 
print(dfFinal[categoricalColumn].isnull().value_counts(),'\n','-'*40)        
    

# ////////////////////// Run this block to drop ***ROWS** with missing values on (---specific column---)  
columnToCheckMissingValues = ['name'] #<------specify name of column here----

print('\n******** ROWS WITH MISSING VALUES ON COLUMN',columnToCheckMissingValues,' WERE DROPPED !','\n','-'*40) 
print('Rows before: ',len(dfFinal),'\n','-'*40) 
dfFinal.dropna(subset=columnToCheckMissingValues,axis=0,inplace=True)
print('Rows now: ',len(dfFinal),'\n','-'*40)  
    
# ////////////////////// Run this block to Drop ****ENTIRE**** column by name (---specify---) 
columnToBeDropped = ['name'] #<------specify column to drop here----

print('Columns before: ',len(dfFinal.columns),dfFinal.columns,'\n','-'*40) 
dfFinal.drop(columnToBeDropped,axis=1,inplace=True)
print('\n******** COLUMN',columnToBeDropped,' WAS DROPPED !','\n','-'*40)
print('Columns now: ',len(dfFinal.columns),dfFinal.columns,'\n','-'*40) 

#/////////////////////////////Run this block to show remaining features and if there are still missing values for them 
print(dfFinal.dtypes)
print('\n','-'*40,'\n','Missing Values\n','-'*40,'\n',dfFinal.isnull().any().value_counts())  

#///////Format correction///////////////Run this block to correct wrong data types of columns (--specify list, specify new data type)
featuresToBeConverted = []#<---specify here Ex: ['population','total_cases','new_cases','new_deaths','total_deaths'] 
newDataType = 'type'#<---specify here Ex: 'int'

print('\n******** DATA TYPES OF',featuresToBeConverted,' WERE CHANGED !','\n','-'*40) 
for feature in featuresToBeConverted:
    print(feature,': converted from',dfFinal[feature].dtype,'to ---->')
    dfFinal[feature] = dfFinal[feature].astype(newDataType)
    print(dfFinal[feature].dtype)
dfFinal.dtypes

#///////Unit Convertion///////////////Run this block to convert units of columns and change their names as well (--specify lists)
featuresToBeConverted = []#<---old column names: Ex: ['total_cases_per_million','new_cases_per_million','new_cases_smoothed_per_million']
newNamesToBeReceived = []#<---new column names: Ex: ['total_cases_per_thousand','new_cases_per_thousand','new_cases_smoothed_per_thousand']

print('\n******** UNITS OF COLUMNS',featuresToBeConverted,' WERE CONVERTED, AND NAMES CHANGED TO',newNamesToBeReceived,'\n','-'*40) 
for feature in zip(featuresToBeConverted,newNamesToBeReceived):
    dfFinal[feature[0]] = #<------ define here the conversion Ex: dfFinal[feature[0]] /=1000 
    dfFinal.rename(columns={feature[0]:feature[1]},inplace=True)
dfFinal.head()

#///////Convention standardisation ///////////////Run this block to standardize a convention for a column (--specify column,old value,new value)
feature= 'name' #<------specify column name Ex:'location' 
value1='old value' #<------specify old value Ex: 'Afghanistan' 
value2='new value' #<------specify new value Ex: 'AFGHANISTAN' 

print('\n******** COLUMN [',feature,'] HAD ITS VALUES CHANGED FROM',value1,'TO',value2,'\n','-'*40) 
dfFinal[feature].replace(value1,value2,inplace=True)
dfFinal.head()


#////////////////////// Run this block to Normalize features 
typeOfNormalization = 0  #<---specify one of these types: (0) Simple Feature Scaling  (1) Min-max  (2) Z-score
featuresToBeNormalized = []#<-----specify features--- Ex:['population_density','life_expectancy']

nameOfNormalization = ['Simple Feature Scaling' ,'Min-max','Z-score']

print('\n******** COLUMNS:',featuresToBeNormalized,' WERE NORMALIZED WITH',nameOfNormalization[typeOfNormalization],'\n','-'*40)  

for feature in featuresToBeNormalized:
    print(feature,': normalized from range [ %.2f , %.2f]' % (dfFinal[feature].min(),dfFinal[feature].max()))

    if  typeOfNormalization == 0:
        dfFinal[feature] = dfFinal[feature] / dfFinal[feature].max()
    elif typeOfNormalization == 1:
        dfFinal[feature] = (dfFinal[feature] - dfFinal[feature].min()) / (dfFinal[feature].max() - dfFinal[feature].min() )  
    else: # typeOfNormalization == (2) Z-score        
        dfFinal[feature] = (dfFinal[feature] - dfFinal[feature].mean()) / dfFinal[feature].std()       

    print('to ----> [ %.2f , %.2f]' % (dfFinal[feature].min(),dfFinal[feature].max()))

